![walmart.jpg](https://i5.walmartimages.com/dfw/4ff9c6c9-37f7/k2-_ec8f9175-e507-46a2-8cdd-cbb03128a52d.v1.svg)

# Background

Walmart is a renowned retail corporation that operates a chain of hypermarkets. Here, Walmart has provided a data combining of 45 stores including store information and monthly sales. The data is provided on weekly basis. Walmart tries to find the impact of holidays on the sales of store. For which it has included four holidays’ weeks into the dataset which are Christmas, Thanksgiving, Super bowl, Labor Day. Here we are owing to Analyze the dataset given. Before doing that, let me point out the objective of this analysis. 

# Business Objectives

Our Main Objective is to predict sales of store in a week. As in dataset size and time related data are given as feature, so analyze if sales are impacted by time-based factors and space- based factor. Most importantly how inclusion of holidays in a week soars the sales in store? 

![sales prediction image.jpg](https://miro.medium.com/max/1400/0*7tM5SbKstuED5_AX.jpg)

# Importing Necessary Libraries and Data

In [ ]:
import numpy as np      # To use np.arrays
import pandas as pd     # To use dataframes

# To plot
import matplotlib.pyplot as plt 
import seaborn as sns

#For date-time
from datetime import datetime

#For model selection
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")



In [ ]:
walmart_store = pd.read_csv('../input/walmart-sales/stores.csv') #store data

In [ ]:
walmart_train = pd.read_csv('../input/walmart-sales/train.csv') # train set

In [ ]:
walmart_features = pd.read_csv('../input/walmart-sales/features.csv') #external information

# First Look to Data and Merging Three Dataframes

In [ ]:
walmart_store.head()

In [ ]:
walmart_train.head()

In [ ]:
walmart_features.head()

In [ ]:
# merging 3 different sets
walmart = walmart_train.merge(walmart_features, on=['Store', 'Date','IsHoliday'], how='inner').merge(walmart_store, on=['Store'], how='inner')
walmart.head(5)

In [ ]:
walmart.head() # last ready data set

In [ ]:
walmart.shape

# Markdown Columns

Walmart gave markdown columns to see the effect if markdowns on sales. When we check columns, there are many NaN values for markdowns. we decided to change them to 0, because if there is markdown in the row, it is shown with numbers. So, if we can write 0, it shows there is no markdown at that date.

In [ ]:
walmart.isna().sum()

In [ ]:
walmart = walmart.fillna(0) # filling null's with 0

In [ ]:
walmart.isna().sum() # last null check

In [ ]:
walmart.describe() # to see weird statistical things

Minimum value for weekly sales is 0.01. Most probably, this value is not true but we prefer not to change them now. Because, there are many departments and many stores. It takes too much time to check each department for each store (45 store for 81 departments). So, we take averages for EDA. 

# Store & Department Numbers

In [ ]:
len(walmart['Store'].unique()) # number of different values

In [ ]:
len(walmart['Dept'].unique()) # number of different values

Now, we will look at the average weekly sales for each store in each department to see if there is any weird values or not. There are 45 stores and 81 departments for stores. 

In [ ]:
store_dept_table = pd.pivot_table(walmart, index='Store', columns='Dept',
                                  values='Weekly_Sales', aggfunc=np.mean)
display(store_dept_table)

Store numbers begin from 1 to 45, department numbers are from 1 to 99, but some numbers are missing such as there is no 88 or 89 etc. Total number of departments is 81. 

From the pivot table, it is obviously seen that there are some wrong values such as there are 0 and minus values for weekly sales. But sales amount can not be minus. Also, it is impossible for one department not to sell anything whole week. So, we will consider the values with sales greater than 0.

In [ ]:
walmart = walmart.loc[walmart['Weekly_Sales'] > 0]

In [ ]:
walmart.shape # new data shape

# Date

In [ ]:
walmart['Date'].head(5).append(walmart['Date'].tail(5)) # to see first and last 5 rows.

Our data is from 5th of February 2010 to 26th of October 2012.  

# IsHoliday column

In [ ]:
sns.barplot(x='IsHoliday', y='Weekly_Sales', data=walmart)

In [ ]:
walmart_holiday = walmart.loc[walmart['IsHoliday']==True]
walmart_holiday['Date'].unique() 

In [ ]:
walmart_not_holiday = walmart.loc[walmart['IsHoliday']==False]
walmart_not_holiday['Date'].nunique()

All holidays are not in the data. There are 4 holiday values such as;

Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13

Labor Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13

Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13

Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13


After the 07-Sep-2012 holidays are in test set for prediction. When we look at the data, average weekly sales for holidays are significantly higher than not-holiday days. In train data, there are 133 weeks for non-holiday and 10 weeks for holiday.

we want to see differences between holiday types. So, we create new columns for 4 types of holidays and fill them with boolean values. If date belongs to this type of holiday it is True, if not False. 

In [ ]:
# Super bowl dates in train set
walmart.loc[(walmart['Date'] == '2010-02-12')|(walmart['Date'] == '2011-02-11')|(walmart['Date'] == '2012-02-10'),'Super_Bowl'] = True
walmart.loc[(walmart['Date'] != '2010-02-12')&(walmart['Date'] != '2011-02-11')&(walmart['Date'] != '2012-02-10'),'Super_Bowl'] = False

In [ ]:
# Labor day dates in train set
walmart.loc[(walmart['Date'] == '2010-09-10')|(walmart['Date'] == '2011-09-09')|(walmart['Date'] == '2012-09-07'),'Labor_Day'] = True
walmart.loc[(walmart['Date'] != '2010-09-10')&(walmart['Date'] != '2011-09-09')&(walmart['Date'] != '2012-09-07'),'Labor_Day'] = False

In [ ]:
# Thanksgiving dates in train set
walmart.loc[(walmart['Date'] == '2010-11-26')|(walmart['Date'] == '2011-11-25'),'Thanksgiving'] = True
walmart.loc[(walmart['Date'] != '2010-11-26')&(walmart['Date'] != '2011-11-25'),'Thanksgiving'] = False

In [ ]:
#Christmas dates in train set
walmart.loc[(walmart['Date'] == '2010-12-31')|(walmart['Date'] == '2011-12-30'),'Christmas'] = True
walmart.loc[(walmart['Date'] != '2010-12-31')&(walmart['Date'] != '2011-12-30'),'Christmas'] = False

In [ ]:
sns.barplot(x='Christmas', y='Weekly_Sales', data=walmart) # Christmas holiday vs not-Christmas

In [ ]:
sns.barplot(x='Thanksgiving', y='Weekly_Sales', data=walmart) # Thanksgiving holiday vs not-thanksgiving

In [ ]:
sns.barplot(x='Super_Bowl', y='Weekly_Sales', data=walmart) # Super bowl holiday vs not-super bowl

In [ ]:
sns.barplot(x='Labor_Day', y='Weekly_Sales', data=walmart) # Labor day holiday vs not-labor day

It is shown that for the graphs, Labor Day and Christmas do not increase weekly average sales. There is positive effect on sales in Super bowl, but the highest difference is in the Thanksgiving. we assume, people generally prefer to buy Christmas gifts 1-2 weeks before Christmas, so it does not change sales in the Christmas week. And, there is Black Friday sales in the Thanksgiving week.

# Type Effect on Holidays

There are three different store types in the data as A, B and C.

In [ ]:
walmart.groupby(['Christmas','Type'])['Weekly_Sales'].mean()  # Avg weekly sales for types on Christmas 

In [ ]:
walmart.groupby(['Labor_Day','Type'])['Weekly_Sales'].mean()  # Avg weekly sales for types on Labor Day

In [ ]:
walmart.groupby(['Thanksgiving','Type'])['Weekly_Sales'].mean()  # Avg weekly sales for types on Thanksgiving

In [ ]:
walmart.groupby(['Super_Bowl','Type'])['Weekly_Sales'].mean()  # Avg weekly sales for types on Super Bowl

In [ ]:
walmart.groupby('IsHoliday')['Weekly_Sales'].mean()     #avg weekly sales on holiday and not on holiday

Nearly, half of the stores are belongs to Type A.

In [ ]:
walmart.sort_values(by='Weekly_Sales',ascending=False).head(5)   #to get which weeks have the highest weekly sales

Also, it is not surprise that top 5 highest weekly sales are belongs to Thanksgiving weeks.

# To See the Size - Type Relation

In [ ]:
walmart_store.groupby('Type').describe()['Size'] # To See the Size-Type relation 

In [ ]:
plt.figure(figsize=(10,8)) # To see the type-size relation
fig = sns.boxplot(x='Type', y='Size', data=walmart)

Size of the type of stores are consistent with sales, as expected. Higher size stores has higher sales. And, Walmart classify stores according to their sizes according to graph. After the smallest size value of Type A, Type B begins. After the smallest size value of Type B, Type C begins.

# Changing Date to Datetime and Creating New Columns

In [ ]:
walmart["Date"] = pd.to_datetime(walmart["Date"]) # convert to datetime
walmart['week'] =walmart['Date'].dt.week
walmart['month'] =walmart['Date'].dt.month 
walmart['year'] =walmart['Date'].dt.year

In [ ]:
walmart.groupby('month')['Weekly_Sales'].mean() # to see the best months for sales

In [ ]:
walmart.groupby('year')['Weekly_Sales'].mean() # to see the best years for sales

In [ ]:
fig = sns.barplot(x='month', y='Weekly_Sales', data=walmart)

When we look at the graph above, the best sales are in December and November, as expected. The highest values are belongs to Thankgiving holiday but when we take average it is obvious that December has the best value.

# Fuel Price, CPI , Unemployment , Temperature Effects

In [ ]:
fuel_price = pd.pivot_table(walmart, values = "Weekly_Sales", index= "Fuel_Price")
fuel_price.plot()

In [ ]:
temp = pd.pivot_table(walmart, values = "Weekly_Sales", index= "Temperature")
temp.plot()

In [ ]:
CPI = pd.pivot_table(walmart, values = "Weekly_Sales", index= "CPI")
CPI.plot()

In [ ]:
unemployment = pd.pivot_table(walmart, values = "Weekly_Sales", index= "Unemployment")
unemployment.plot()

From graphs, it is seen that there are no significant patterns between CPI, temperature, unemployment rate, fuel price vs weekly sales.

In [ ]:
walmart.to_csv('clean_data.csv') # assign new data frame to csv for using after here

# Findings and Explorations

- There are 45 stores and 81 department in data. Departments are not same in all stores.
- Stores has 3 types as A, B and C according to their sizes. Almost half of the stores are bigger than 150000 and categorized as A. According to type, sales of the stores are changing.
- As expected, holiday average sales are higher than normal dates.
- Thankgiving has higher sales  which was among all the other weeks assigned by Walmart.
- January sales are significantly less than other months. This is the result of November and December high sales. After two high sales month, people prefer to pay less on January.
- CPI, temperature, unemployment rate and fuel price have no pattern on weekly sales. 


# Random Forest Regression

In [ ]:
df = pd.read_csv('clean_data.csv')

In [ ]:
df['Date'] = pd.to_datetime(df['Date']) # changing datetime to divide if needs

# Encoding the Data 

For preprocessing our data, we will change holidays boolean values to 0-1 and replace type of the stores from A, B, C to 1, 2, 3. 

In [ ]:
df_encoded = df.copy() # to keep original dataframe taking copy of it

In [ ]:
type_group = {'A':1, 'B': 2, 'C': 3}  # changing A,B,C to 1-2-3
df_encoded['Type'] = df_encoded['Type'].replace(type_group)

In [ ]:
df_encoded['Super_Bowl'] = df_encoded['Super_Bowl'].astype(bool).astype(int) # changing T,F to 0-1

In [ ]:
df_encoded['Thanksgiving'] = df_encoded['Thanksgiving'].astype(bool).astype(int) # changing T,F to 0-1

In [ ]:
df_encoded['Labor_Day'] = df_encoded['Labor_Day'].astype(bool).astype(int) # changing T,F to 0-1

In [ ]:
df_encoded['Christmas'] = df_encoded['Christmas'].astype(bool).astype(int) # changing T,F to 0-1

In [ ]:
df_encoded['IsHoliday'] = df_encoded['IsHoliday'].astype(bool).astype(int) # changing T,F to 0-1

In [ ]:
df_new = df_encoded.copy() # taking the copy of encoded df to keep it original

# Observation of Interactions between Features

Firstly, we will drop divided holiday columns from my data and try without them. To keep my encoded data safe, we assigned my dataframe to new one and will use. 

In [ ]:
drop_col = ['Super_Bowl','Labor_Day','Thanksgiving','Christmas']
df_new.drop(drop_col, axis=1, inplace=True) # dropping columns

In [ ]:
plt.figure(figsize = (12,10))
sns.heatmap(df_new.corr().abs())    # To see the correlations
plt.show()

Temperature, unemployment, CPI have no significant effect on weekly sales, so we will drop them. Also, Markdown 4 and 5 highly correlated with Markdown 1. So, we will drop them too. It can create multicollinearity problem, maybe. So, we will work without them.

In [ ]:
drop_col = ['Temperature','MarkDown4','MarkDown5','CPI','Unemployment']
df_new.drop(drop_col, axis=1, inplace=True) # dropping columns

In [ ]:
plt.figure(figsize = (12,10))
sns.heatmap(df_new.corr().abs())    # To see the correlations without dropping columns
plt.show()

In [ ]:
df_new = df_new.sort_values(by='Date', ascending=True) # sorting according to date

# Creating Train-Test Splits

Our date column has continuous values, to keep the date features continuous, we will not take random splitting. so, we split data manually according to 70%.

In [ ]:
train_data = df_new[:int(0.7*(len(df_new)))] # taking train part
test_data = df_new[int(0.7*(len(df_new))):] # taking test part

target = "Weekly_Sales"
used_cols = [c for c in df_new.columns.to_list() if c not in [target]] # all columns except weekly sales

X_train = train_data[used_cols]
X_test = test_data[used_cols]
y_train = train_data[target]
y_test = test_data[target]

In [ ]:
X = df_new[used_cols] # to keep train and test X values together

We have enough information as we split the date such as week of the year,month etc. So, we drop date columns.

In [ ]:
X_train = X_train.drop(['Date'], axis=1) # dropping date from train
X_test = X_test.drop(['Date'], axis=1) # dropping date from test

# Random Forest Regressor

To tune the regressor, I can use gridsearch but it takes too much time for this type of data which has many rows and columns. So,we choose regressor parameters manually. we changed the parameters each time and try to find the best result.

In [ ]:
rf = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1, max_depth=35,
                           max_features = 'sqrt',min_samples_split = 10).fit(X_train, y_train)


In [ ]:
prediction = rf.predict(X_test)
prediction

In [ ]:
y_test

In [ ]:
mse = mean_squared_error(y_test, prediction).round(2)
rmse = (mse**.5).round(2)
print(mse)
print(rmse)